AIMessage(content='AI:\n영화이름: Zootopia 2\n감독: Byron Howard, Rich Moore\n주요출연진: Ginnifer Goodwin, Jason Bateman\n예산: 180,000,000 USD\n흥행수익: 1,000,000,000 USD\n장르: 애니메이션 / 어드벤처\n시놉시스: 주토피아에서 새로운 사건이 발생하며 주인공 주디와 닉이 다시 모험에 뛰어들게 되는 이야기를 그린 애니메이션 영화.')